In [89]:
import numpy as np

In [90]:
#body_xyz_data = np.load('./recording_19_56_30_gmt-4/output_data/mediapipe_body_3d_xyz.npy')
#left_hand_xyz_data = np.load('./recording_19_56_30_gmt-4/output_data/mediapipe_left_hand_3d_xyz.npy')
#right_hand_xyz_data = np.load('./recording_19_56_30_gmt-4/output_data/mediapipe_right_hand_3d_xyz.npy')

In [91]:
# get max distance between left and right hand. this will be the approximated wingspan.

# get difference between max left and max right when left is farthest left and right is farthest right. the one with the greatest distance wins
#left_max = np.max(left_hand_xyz_data[:, :, 0])  # min since 0,0 starts at top-left
#left_max_index = np.where(left_hand_xyz_data == left_max)
#left_max_wingspan = left_hand_xyz_data[left_max_index[0], left_max_index[1], 2][0] - left_max
#print(left_max_wingspan)
#right_max = np.max(right_hand_xyz_data[:, :, 2])  # min since 0,0 starts at top-right
#right_max_index = np.where(right_hand_xyz_data == right_max)
#right_max_wingspan = right_hand_xyz_data[right_max_index[0], right_max_index[1], 0][0] - right_max
#print(right_max_wingspan)

### Taken from example .ipynb

In [92]:
from pathlib import Path

try:
    import numpy as np
except Exception as e:
    print(e)
    %pip install numpy
    import numpy as np


try:
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go
except Exception as e:
    print(e)
    %pip install plotly
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go
    

In [93]:
# function to get data for particular body part
def get_bodypart_data(bodypart = "left_index"):
    path_to_recording = "C:\\Users\\briid\\Documents\\Research\\Arm-Simulation-with-Forces\\experimentation\\freemocap\\recording_19_56_30_gmt-4"
    mediapipe_indices = ['nose',
    'left_eye_inner',
    'left_eye',
    'left_eye_outer',
    'right_eye_inner',
    'right_eye',
    'right_eye_outer',
    'left_ear',
    'right_ear',
    'mouth_left',
    'mouth_right',
    'left_shoulder',
    'right_shoulder',
    'left_elbow',
    'right_elbow',
    'left_wrist',
    'right_wrist',
    'left_pinky',
    'right_pinky',
    'left_index',
    'right_index',
    'left_thumb',
    'right_thumb',
    'left_hip',
    'right_hip',
    'left_knee',
    'right_knee',
    'left_ankle',
    'right_ankle',
    'left_heel',
    'right_heel',
    'left_foot_index',
    'right_foot_index']

    joint_to_plot_index = mediapipe_indices.index(bodypart)


    path_to_recording = Path(path_to_recording)
    path_to_center_of_mass_npy = path_to_recording/'output_data'/'center_of_mass'/'total_body_center_of_mass_xyz.npy'
    path_to_freemocap_3d_body_data_npy = path_to_recording/'output_data'/'mediapipe_body_3d_xyz.npy'

    freemocap_3d_body_data = np.load(path_to_freemocap_3d_body_data_npy)
    total_body_com_data = np.load(path_to_center_of_mass_npy)

    return freemocap_3d_body_data[:,joint_to_plot_index,:]

#### Experimenting:

In [94]:
# get data for both hands
left_hand_data = get_bodypart_data("left_wrist")
right_hand_data = get_bodypart_data("right_wrist")
# and index fingers
left_index_data = get_bodypart_data("left_index")
right_index_data = get_bodypart_data("right_index")

In [95]:
# function to get distance between two body parts
# parameters are the data arrays for the body parts
# returns array of distances between parts
def dist_between_vertices(first_part, second_part):
    cur_dist = np.ndarray(shape = first_part.shape, dtype = first_part.dtype)
    for i in range(0, len(first_part) - 1):     # get the distance for each frame of the video (excluding last frame)
        cur_dist[i] = np.linalg.norm(first_part[i] - second_part[i])
        #print(cur_dist[i])]
    return cur_dist[:-1, 0]

In [96]:
dist = dist_between_vertices(left_index_data, right_index_data)

In [97]:
# find largest distance between both hands
np.max(dist)

921.5067001657222

In [98]:
# get ratio to real distance in meters
real_height_metric = 1.78     # meters
sim_wingspan = np.max(dist)     # max distance between wrists via `freemocap`

sim_to_real_conversion_factor = real_height_metric / sim_wingspan
print(sim_to_real_conversion_factor)

0.0019316191620526338


In this case, we have acquired a sim-to-real (metric) conversion factor of approximately `0.0018990637829`.

For a roughly approximate method of checking the validity of the conversion factor, we will calculate the distance from the eyes (`left_eye` and `right_eye`) to the heels (`left_heel` and `right_heel`).

***Keep in mind, the full body was not in frame for this run at any point, so this estimate is going to be off by a decent margin. This was done for experimentation purposes only.***

In [99]:
# get data for eyes and heels
left_eye_data = get_bodypart_data("left_eye")
right_eye_data = get_bodypart_data("right_eye")
left_heel_data = get_bodypart_data("left_heel")
right_heel_data = get_bodypart_data("right_heel")

In [100]:
# calculate distance for each side, then divide by 2 to get the average of the two
height_approx_left = np.max(dist_between_vertices(left_eye_data, left_heel_data))
height_approx_right = np.max(dist_between_vertices(right_eye_data, right_heel_data))
rough_approx_height_sim = (height_approx_left + height_approx_right) / 2

In [101]:
rough_approx_height = rough_approx_height_sim * sim_to_real_conversion_factor
print("%s meters" % rough_approx_height)

2.379298277951489 meters


Not great, but only a little less than 25% off from the real value, so not too terrible. Let's try something more reasonable, like the length of one arm.

In [102]:
left_shoulder_data = get_bodypart_data("left_shoulder")
right_shoulder_data = get_bodypart_data("right_shoulder")
np.max(left_shoulder_data)

803.6215345400643

In [103]:
# right hand to shoulder
print("%s meters" % (np.max(dist_between_vertices(right_shoulder_data, right_index_data)) * sim_to_real_conversion_factor))

0.9283948802129095 meters


In [104]:
# left hand to shoulder
print("%s meters" % (np.max(dist_between_vertices(left_index_data, left_shoulder_data)) * sim_to_real_conversion_factor))

0.8022724104020942 meters


### Trying to do depth:

- try to do this by calculating the ratio between the current length of any given body part and its longest length.
    - the shorter a part is as compared to its longest length, the more depth needs to be accounted for. 
    - a distance of 0 between the start and end of a body part segment would mean it is directly in line with the camera. a distance of length N where N is the median of the 5 longest measurements of length between two points (i.e. elbow and wrist), should show that the segment is normal to the angle of the camera (thus no depth).
- to figure out which way the depth goes (forwards or back), calculate the difference in light intensity between the vertices in question. whichever one is brighter is closer.
    - could also see whether it's getting brighter or darker over time, which should probably help, I think

In [105]:
# taken from the default output .ipynb
bodypart = 'left_index'
path_to_recording = "C:\\Users\\briid\\Documents\\Research\\Arm-Simulation-with-Forces\\experimentation\\freemocap\\recording_19_56_30_gmt-4"
mediapipe_indices = ['nose',
'left_eye_inner',
'left_eye',
'left_eye_outer',
'right_eye_inner',
'right_eye',
'right_eye_outer',
'left_ear',
'right_ear',
'mouth_left',
'mouth_right',
'left_shoulder',
'right_shoulder',
'left_elbow',
'right_elbow',
'left_wrist',
'right_wrist',
'left_pinky',
'right_pinky',
'left_index',
'right_index',
'left_thumb',
'right_thumb',
'left_hip',
'right_hip',
'left_knee',
'right_knee',
'left_ankle',
'right_ankle',
'left_heel',
'right_heel',
'left_foot_index',
'right_foot_index']

joint_to_plot_index = mediapipe_indices.index(bodypart)


path_to_recording = Path(path_to_recording)
path_to_center_of_mass_npy = path_to_recording/'output_data'/'center_of_mass'/'total_body_center_of_mass_xyz.npy'
path_to_freemocap_3d_body_data_npy = path_to_recording/'output_data'/'mediapipe_body_3d_xyz.npy'

freemocap_3d_body_data = np.load(path_to_freemocap_3d_body_data_npy)
total_body_com_data = np.load(path_to_center_of_mass_npy)

# save pre-modification for testing purposes
body_data_pre_mod = freemocap_3d_body_data

In [106]:
# get indices for body parts
def get_index(body_part = 'left_wrist'):
    return mediapipe_indices.index(body_part)

In [107]:
# function to get data for particular body part
def get_bodypart_data(bodypart = "left_index"):
    
    joint_to_plot_index = mediapipe_indices.index(bodypart)

    return freemocap_3d_body_data[:,joint_to_plot_index,:]

In [108]:
# function to get distance between two body parts
# parameters are the data arrays for the body parts
# returns array of distances between parts
def dist_between_vertices(first_part, second_part):
    cur_dist = np.ndarray(shape = first_part.shape, dtype = first_part.dtype)
    for i in range(0, len(first_part) - 1):     # get the distance for each frame of the video (excluding last frame)
        cur_dist[i] = np.linalg.norm(first_part[i] - second_part[i])
        #print(cur_dist[i])]
    return cur_dist[:-1, 0]

In [109]:
# get body part data
left_index_data = get_bodypart_data("left_index")
left_elbow_data = get_bodypart_data("left_elbow")

In [110]:
# ratio between current length and zero length:
left_hand_to_elbow_array = dist_between_vertices(left_index_data, left_elbow_data)

In [111]:
# get median of largest distances between vertices/bodyparts
def max_dist_between_parts(dist_array):
    ind = (np.argpartition(dist_array, -20)[-20:-5])
    return np.median(dist_array[ind])

In [112]:
# get approximate distance between vertices/body parts
left_hand_to_elbow_dist = max_dist_between_parts(left_hand_to_elbow_array)
print(left_hand_to_elbow_dist)

290.24718732016095


In [113]:
# calculate the angle of the segment (body part) from the normal (where it is longest)
def angle_from_normal(cur_dist, max_dist):
    return np.arccos(cur_dist / max_dist)

In [114]:
angle_from_normal(0, left_hand_to_elbow_dist)

1.5707963267948966

Nice! The output, `1.570796`, is half of pi, or pi/2. This is exactly what we were looking for when the cur_dist is 0, telling us that it works as expected.

Now, use this to calculate the distance the (body part) vertex is away from the norm with the angle, and there's your depth.

In [115]:
test_vertex_one = get_bodypart_data('left_index')
test_vertex_two = get_bodypart_data('left_elbow')
test_dist_array = dist_between_vertices(test_vertex_one, test_vertex_two)

test_dist_array = test_dist_array
test_max_dist = max_dist_between_parts(test_dist_array)
test_angle = angle_from_normal(test_dist_array[13], test_max_dist)
test_depth = np.sin(test_angle) * test_max_dist
print(test_angle)
print(test_depth)

0.6537383962511865
176.5162205135117


In [116]:
# get depth
#def get_depth(vertex_one, vertex_two, timestep):
#    dist_array = dist_between_vertices(vertex_one, vertex_two)
#    max_dist = max_dist_between_parts(dist_array)
#    cur_dist = dist_array[timestep]
#    angle = angle_from_normal(cur_dist, max_dist)
#    depth = np.sin(angle) * max_dist
#    return depth

In [117]:
# get depth
def get_depth(vertex_one, vertex_two):
    dist_array = dist_between_vertices(vertex_one, vertex_two)
    max_dist = max_dist_between_parts(dist_array)
    depths = list()
    for frame in dist_array:
        angle = angle_from_normal(frame, max_dist)
        depths.append(np.sin(angle) * max_dist)
    return depths

In [118]:
np.nan_to_num(get_depth(get_bodypart_data('left_index'), get_bodypart_data('left_elbow')))#, 10))

C:\Users\briid\AppData\Local\Temp\ipykernel_9456\1837775029.py:3: RuntimeWarning:

invalid value encountered in arccos



array([116.751, 194.749, 218.095, 212.428, 190.816, 169.001, 171.727,
       201.447, 225.075, 226.758, 211.327, 198.137, 193.619, 176.516,
       150.386, 187.945, 260.932, 286.754, 278.931, 237.342, 195.326,
       200.242, 207.6  , 198.379, 188.51 , 191.073, 201.165, 208.95 ,
       209.746, 203.17 , 192.327, 182.265, 174.382, 166.66 , 159.368,
       156.064, 158.986, 166.182, 173.616, 178.103, 179.457, 180.544,
       183.502, 186.918, 188.832, 189.638, 190.453, 191.339, 192.369,
       194.14 , 196.264, 197.993, 200.867, 207.163, 215.913, 223.983,
       229.599, 232.248, 231.044, 225.771, 219.218, 215.763, 216.11 ,
       216.967, 216.281, 215.102, 214.986, 215.929, 217.304, 219.035,
       220.919, 221.96 , 221.407, 219.919, 218.891, 218.725, 218.471,
       217.537, 216.888, 217.848, 220.261, 222.91 , 225.224, 227.369,
       229.046, 229.48 , 228.537, 227.112, 226.633, 227.969, 230.084,
       230.352, 227.327, 223.04 , 221.356, 223.246, 225.7  , 225.981,
       224.832, 224.

Test it out by running it for each time step then setting the values for `y` in the data output matrices to the calculated values with `get_depth`. 

Visualize in the same way it does by default in order to check how well it works.

In [119]:
# do depth for each time set, set for each vertex according to its nearest neighbor

In [120]:
# put together pairs for each of the vertices
# the first one is the point which will be moved on the y-axis, the second one for calculations
#vertex_pairs = [    # this set only for upper body
#    ['left_shoulder', 'right_shoulder'],
#    ['left_elbow', 'left_shoulder'],
#    ['left_index', 'left_elbow'],
#    ['right_elbow', 'right_shoulder'],
#    ['right_index','right_elbow']
#]
vertex_order = [
    [
        'left_shoulder',
        'right_shoulder'
    ],
    [
        'left_shoulder',
        'left_elbow',
        'left_wrist',
    ],
    [
        'right_shoulder',
        'right_elbow',
        'right_wrist'
    ]
]

In [121]:
# get length of body parts by vertex pairs
#def get_y_axes(vertex_pairs = vertex_pairs):
#    lengths = list()
#    for vertex_pair in vertex_pairs:
#        lengths.append(np.nan_to_num(get_depth(get_bodypart_data(vertex_pair[0]), get_bodypart_data(vertex_pair[1]))))
#    return lengths

# get y axes by order of body parts
def get_y_axes(vertex_order = vertex_order):
    y = list()
    for vertices in vertex_order:
        group_y = list()
        num_vertices = len(vertices)
        for i, vertex in enumerate(vertices):
            if i < (num_vertices - 1):
                y_dist_between_vertices = np.nan_to_num(get_depth(get_bodypart_data(vertices[i]), get_bodypart_data(vertices[i + 1])))
                if i > 0:
                    vertex_y = group_y[i - 1] +  y_dist_between_vertices    # add y depth of anchor to current
                else:
                    vertex_y = y_dist_between_vertices
                group_y.append(vertex_y)
        y.append(group_y)
    return y



In [122]:
# test get_y_axes
#print(np.asarray(get_y_axes()).shape)
print(get_y_axes())


[[array([139.272, 135.649, 136.562, 142.768, 150.26 , 154.75 , 154.994,
       153.108, 152.987, 157.603, 165.986, 173.166, 175.202, 172.952,
       170.254, 168.763, 166.35 , 161.533, 156.469, 153.941, 153.245,
       151.533, 147.196, 140.681, 134.778, 133.829, 138.503, 143.548,
       144.382, 141.682, 138.8  , 138.035, 141.042, 148.659, 157.929,
       163.851, 164.779, 163.821, 164.784, 168.374, 174.04 , 182.272,
       192.152, 200.095, 203.583, 203.859, 203.937, 204.652, 204.158,
       201.101, 196.43 , 191.89 , 187.924, 184.354, 181.9  , 181.053,
       181.019, 181.511, 183.362, 186.201, 188.226, 188.972, 189.891,
       191.93 , 194.227, 195.571, 195.756, 195.273, 194.53 , 193.707,
       192.948, 192.242, 191.213, 189.349, 186.679, 184.223, 183.384,
       184.273, 185.053, 183.885, 181.28 , 179.516, 179.409, 179.3  ,
       177.751, 175.354, 173.575, 172.958, 172.925, 172.783, 172.56 ,
       172.975, 174.496, 176.483, 177.847, 178.423, 178.938, 179.45 ,
       178.784, 17

C:\Users\briid\AppData\Local\Temp\ipykernel_9456\1837775029.py:3: RuntimeWarning:

invalid value encountered in arccos



In [123]:
# get y axes
y_axes = get_y_axes()
# account for difference between shoulder y-axes
y_axes[2] += y_axes[0]  # by adding it to the branch off the right shoulder

C:\Users\briid\AppData\Local\Temp\ipykernel_9456\1837775029.py:3: RuntimeWarning:

invalid value encountered in arccos



In [124]:
# put together dictionary coordinating everything
depth_dict = {
    'vertex_order': vertex_order,                # pairs of body parts/segments
    'y_axes': y_axes,   # approximated depth
}

### Bringing it all together now:

- current setup uses left shoulder as temp anchor

In [125]:
freemocap_3d_body_data[20, :, 1]    # the '1' here indicates the y-axis

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [126]:
# get indices for body parts used
indices = list()
for vertices in depth_dict['vertex_order']:
    for vertex in vertices:
        indices.append(get_index(vertex))

print(indices)

[11, 12, 11, 13, 15, 12, 14, 16]


for all in dict:
    for all vertices:
        set y to y_axis_depth + y_axis_depth(vertex_anchor)     <--vertex_anchor referring to other vertex, i.e. left_shoulder for left_elbow

In [127]:
# approximate depth for the video recording
def set_depth(depth_dict = depth_dict, body_data = freemocap_3d_body_data):
    depth_body_data = body_data[:, :, 1]    # y axis of each part on each frame

    # go through and set y-axes values accordingly
    for i, order_group in enumerate(depth_dict['y_axes']):
        cur_length = len(depth_dict['vertex_order'][i])
        # go thru all vertices in current group
        for j, vertex in enumerate(order_group):
            if j < (cur_length - 1):
                # set y axis for each vertex in the order group
                cur_vertex = depth_dict['vertex_order'][i][j + 1]   # + 1 so that it applies to the non-anchor vertex
                vertex_index = mediapipe_indices.index(cur_vertex)
                body_data[:, vertex_index, 1] = np.append(vertex, 0)

    body_data[:, :, 1] = depth_body_data
    return body_data

In [128]:
# testing...
test_body_data = freemocap_3d_body_data

test_body_data = set_depth(body_data = test_body_data)

In [129]:
print(test_body_data[:, 13, 1])

[124.269   0.      0.     81.947 124.509 134.833 134.307 136.798 143.096
 146.671 144.54  139.315 136.953 139.476 140.854 133.956 115.584  96.651
  97.105 106.111 106.789 102.344 101.131 101.502  98.42   91.915  84.538
  78.904  77.876  81.211  85.147  88.84   95.081 103.627 110.308 112.486
 111.327 110.441 112.204 115.75  118.943 120.753 121.559 122.441 124.036
 125.676 126.495 126.439 125.904 125.597 126.878 130.737 136.171 141.156
 144.792 147.15  147.935 147.08  146.501 148.777 153.114 155.971 155.451
 152.614 149.814 148.502 148.512 149.037 149.545 149.897 150.112 150.244
 150.336 150.27  149.724 148.611 147.348 146.5   146.562 147.656 149.031
 149.508 148.555 146.695 145.11  144.75  145.482 146.323 146.707 146.986
 147.413 147.398 146.277 144.265 142.309 140.802 139.337 138.214 138.706
 141.121 143.65  144.204 142.862 141.381 140.712 140.485 140.81  142.012
 142.37  140.392 138.035 137.768 140.021 144.094 147.919 148.462 144.873
 139.844 135.714 131.865 130.001 135.172 145.394 15

In [130]:
# set the depth:
freemocap_3d_body_data = set_depth(body_data = freemocap_3d_body_data)

In [131]:
freemocap_3d_body_data[:, 13, 1]

array([124.269,   0.   ,   0.   ,  81.947, 124.509, 134.833, 134.307,
       136.798, 143.096, 146.671, 144.54 , 139.315, 136.953, 139.476,
       140.854, 133.956, 115.584,  96.651,  97.105, 106.111, 106.789,
       102.344, 101.131, 101.502,  98.42 ,  91.915,  84.538,  78.904,
        77.876,  81.211,  85.147,  88.84 ,  95.081, 103.627, 110.308,
       112.486, 111.327, 110.441, 112.204, 115.75 , 118.943, 120.753,
       121.559, 122.441, 124.036, 125.676, 126.495, 126.439, 125.904,
       125.597, 126.878, 130.737, 136.171, 141.156, 144.792, 147.15 ,
       147.935, 147.08 , 146.501, 148.777, 153.114, 155.971, 155.451,
       152.614, 149.814, 148.502, 148.512, 149.037, 149.545, 149.897,
       150.112, 150.244, 150.336, 150.27 , 149.724, 148.611, 147.348,
       146.5  , 146.562, 147.656, 149.031, 149.508, 148.555, 146.695,
       145.11 , 144.75 , 145.482, 146.323, 146.707, 146.986, 147.413,
       147.398, 146.277, 144.265, 142.309, 140.802, 139.337, 138.214,
       138.706, 141.

### Integrating the functions from previous iterations of the project (pt 1: angles and stuff):

In [132]:
# set aside an untouched version of freemocap_3d_body_data for testing purposes
body_data = body_data_pre_mod

#### IMPORTANT: set a bias on all coordinates, so that the left shoulder is [0, 0, 0].

In [133]:
# ONLY RUN THIS ONCE
# # set bias for all frames and body parts so that left shoulder is 0, 0, 0 on the coordinate plane
#for i in range(0, len(freemocap_3d_body_data[0, :, 0])):
#    freemocap_3d_body_data[:, i, :] -= freemocap_3d_body_data[:, 11, :]

#### Getting spherical coordinates from rectangular:

In [134]:
#body_data_no_y = freemocap_3d_body_data[:, :, :]
#body_data_no_y[:, :, 1] = 0
#print(body_data_no_y[:, 13, 1])

In [135]:
# starting point
#body_data = freemocap_3d_body_data
#save_body_data = body_data

In [136]:
#body_data = save_body_data

##### Normalization

Getting data with `left_shoulder` as frame of reference (0, 0, 0 on the rectangular coordinate plane)

In [137]:
# get into negative number range for all values
#for i in range(0, len(body_data)):
#    for vertex in [11, 13, 15]:
#        body_data[i, vertex, 0] -= 1000     # subtract from x
#        body_data[i, vertex, 1] -= 1000     # subtract from y

In [138]:
# set elbow to be 0, 0, 0 (for calculating spherical coordinates)
# go thru and subtract left_elbow from each of the vertices of interest
#for i in range(0, len(body_data)):
#    for vertex in [11, 13, 15]:
#        # set left_shoulder to be 0, 0, 0
#        body_data[i, vertex, :] -= body_data[i, 11, :]

# only run this once

In [139]:
body_data[:, 15, ]

array([[ 752.324,  124.269, -796.412],
       [ 753.103,  117.364, -796.37 ],
       [ 747.646,  141.111, -786.657],
       ...,
       [ 711.6  ,  296.474, -433.098],
       [ 716.762,  276.816, -432.028],
       [ 724.535,    0.   , -443.815]])

As for the others, one thing we can do is normalize the data between max length and 0 for each of the segments (body parts), as follows:

In [140]:
print(freemocap_3d_body_data[:, 11, 0])

[733.896 732.315 728.299 721.179 712.053 702.364 693.067 684.793 677.543
 670.461 663.454 658.595 658.018 660.804 663.986 666.024 666.986 666.126
 662.258 656.513 652.121 650.928 651.749 652.114 650.307 646.077 640.589
 635.402 631.038 627.15  623.816 621.157 617.987 612.914 606.872 602.687
 601.89  603.347 605.073 606.141 606.601 606.616 606.4   606.475 607.208
 608.223 608.896 609.292 609.843 609.984 608.16  603.764 598.226 593.426
 589.676 585.735 579.975 571.479 561.234 552.281 547.263 545.647 544.429
 541.62  538.026 535.568 534.969 535.555 536.473 537.339 537.985 538.27
 538.408 539.096 540.736 542.565 543.259 542.731 542.749 545.043 549.059
 552.599 554.586 555.988 557.928 560.163 561.758 562.362 562.446 562.664
 563.093 563.036 561.817 559.915 558.707 558.775 559.226 559.232 559.383
 560.687 562.927 564.857 565.571 565.013 563.656 562.207 561.262 560.955
 561.175 561.963 563.231 564.372 564.766 564.667 565.072 566.53  568.44
 569.941 571.126 572.357 572.615 570.385 566.408 563.

In [141]:
# get max length of segments/body parts
upper_arm_max_length = max_dist_between_parts(dist_between_vertices(freemocap_3d_body_data[:, 11, :], freemocap_3d_body_data[:, 13, :]))
lower_arm_max_length = max_dist_between_parts(dist_between_vertices(freemocap_3d_body_data[:, 13, :], freemocap_3d_body_data[:, 15, :]))
print(upper_arm_max_length * sim_to_real_conversion_factor)
print(lower_arm_max_length * sim_to_real_conversion_factor)

0.38972487879800544
0.4104332541380157


#### Experimenting

In [142]:
# x,y,z --> rho,theta,phi
#x = body_data[:, 11:17:2, 0]
#y = body_data[:, 11:17:2, 1]
#z = body_data[:, 11:17:2, 2]
#
#rho = np.sqrt(x ** 2 + y ** 2 + z ** 2)
#theta = np.arctan2(y, x)
#phi = np.arccos(z / rho)

In [143]:
body_data[:, 11, :]

array([[ 733.896,    0.   , -423.481],
       [ 732.315,    0.   , -424.27 ],
       [ 728.299,    0.   , -425.09 ],
       ...,
       [ 786.92 ,    0.   , -428.316],
       [ 793.3  ,    0.   , -430.652],
       [ 803.622,    0.   , -439.33 ]])

In [144]:
#rho[:, 0]

In [145]:
# get spherical coordinates from rectangular coordinates
#def get_spher_coord(body_data = freemocap_3d_body_data[:, 13, :]):
#    # normalize coords
#    body_data -= freemocap_3d_body_data[:, 11, :]   # using left_shoulder as 0, 0, 0/point of origin
#
#    x = body_data[:, 0]
#    y = body_data[:, 1]
#    z = body_data[:, 2]
#
#    rho = np.sqrt(x **2 + y ** 2 + z ** 2)
#    theta = np.arctan2(y, x)
#    phi = np.arccos(z / rho)
#    return [rho, theta, phi]

In [146]:
# set last few coords to spherical coords for displaying in graph
#freemocap_3d_body_data[:, 30, :] = np.swapaxes(get_spher_coord(freemocap_3d_body_data[:, 11, :]), 0, 1)   # left shoulder
#freemocap_3d_body_data[:, 31, :] = np.swapaxes(get_spher_coord(freemocap_3d_body_data[:, 13, :]), 0, 1)   # left elbow
#freemocap_3d_body_data[:, 32, :] = np.swapaxes(get_spher_coord(freemocap_3d_body_data[:, 15, :]), 0, 1)   # left wrist


In [147]:
# not scientific notation
np.set_printoptions(suppress = True, precision = 3)

In [148]:

    # add text showing spherical coords -BD
    #text = "Spherical coordinates:\n" +
    #       "\tLeft shoulder: " + str(freemocap_3d_body_data[i, 30, :]) +
    #       "\n\tLeft elbow: " + str(freemocap_3d_body_data[i, 31, :]) +
    #       "\n\tLeft wrist: " + str(freemocap_3d_body_data[i, 32, :])

##### Get angle between upper and lower arm:

In [149]:
# get angle between two segments
#def get_angle_between(bodypart_one, bodypart_two)
#    sim_to_real_conversion_factor

#### Experimenting but again

#### Getting angle at elbow

In [150]:
# get x y and z for spherical coordinate transformations/angle getting
x = body_data[:, 11:17:2, 0]
y = body_data[:, 11:17:2, 1]
z = body_data[:, 11:17:2, 2]

In [151]:
# calculate vectors for getting angle
vector_a = [(x[:, 0] - x[:, 1]), (y[:, 0] - y[:, 1]), (z[:, 0] - z[:, 1])]
vector_b = [(x[:, 2] - x[:, 1]), (y[:, 2] - y[:, 1]), (z[:, 2] - z[:, 1])]

forearm_length = np.sqrt( (vector_b[0] ** 2) + (vector_b[1] ** 2) + (vector_b[2] ** 2) )
upperarm_length = np.sqrt( (vector_a[0] ** 2) + (vector_a[1] ** 2) + (vector_a[2] ** 2) )

angle_between = np.arccos( ( (vector_a[0] * vector_b[0]) + (vector_a[1] * vector_b[1]) + (vector_a[2] * vector_b[2]) ) / (forearm_length * upperarm_length) )

Now put it in the data matrix for display by plotly

In [152]:
freemocap_3d_body_data[:, 30, :] = np.swapaxes(vector_a, 0, 1)
freemocap_3d_body_data[:, 31, :] = np.swapaxes(vector_b, 0, 1)

freemocap_3d_body_data[:, 32, 0] = forearm_length * sim_to_real_conversion_factor
freemocap_3d_body_data[:, 32, 1] = upperarm_length * sim_to_real_conversion_factor
freemocap_3d_body_data[:, 32, 2] = angle_between

#### Getting spherical coordinates

In [153]:
# get spherical coordinates for each of the 3 vertices (bodyparts) of interest, 
#   and set them to overwrite parts 27-29 of freemocap_3d_body_data for displaying with `plotly`

# initialize to x.shaped ndarray since x is conveniently the same shape we want
#rho = np.ndarray(x.shape)
#theta = np.ndarray(x.shape)
#phi = np.ndarray(x.shape)

# set for [shoulder, elbow, wrist]
#for i in range(0, len(rho)):
#    for vertex in [0, 1, 2]:
#        cur_bodypart = freemocap_3d_body_data[i, (11 + vertex * 2), :]
#        rho[i, vertex] = np.sqrt( (cur_bodypart[0] ** 2) + (cur_bodypart[1] ** 2) + (cur_bodypart[2] ** 2) )
#        theta[i, vertex] = np.arctan2(cur_bodypart[1], cur_bodypart[0])
#        phi[i, vertex] = np.arccos( (cur_bodypart[2] / np.sqrt(rho[i, vertex])) )

In [164]:
# get spherical coordinates for each of the 3 vertices (bodyparts) of interest, 
#   and set them to overwrite parts 27-29 of freemocap_3d_body_data for displaying with `plotly`

# initialize to x.shaped ndarray since x is conveniently the same shape we want
rho = np.zeros(x.shape)
theta = np.zeros(x.shape)
phi = np.zeros(x.shape)

# set for [elbow, wrist]    using difference between current point and shoulder point
for i in range(0, len(rho)):
    for vertex in [1, 2]:       # using shoulder just returns 0 bc its 0 from the point of origin which is the shoulder (hence it's missing here)
        if vertex == 1:     # if elbow (from shoulder to elbow)
            cur_shoulder = freemocap_3d_body_data[i, 11, :]
        elif vertex == 2:   # if wrist (from elbow to wrist)
            cur_shoulder = freemocap_3d_body_data[i, 13, :]
        cur_bodypart = freemocap_3d_body_data[i, (11 + vertex * 2), :]
        x_diff = cur_bodypart[0] - cur_shoulder[0]
        y_diff = cur_bodypart[1] - cur_shoulder[1]
        z_diff = cur_bodypart[2] - cur_shoulder[2]
        rho[i, vertex] = np.abs((x_diff ** 2) + (y_diff ** 2) + (z_diff ** 2))
        theta[i, vertex] = np.arctan2(y_diff, x_diff)
        phi[i, vertex] = np.arccos( (z_diff / np.sqrt(rho[i, vertex])) )    # needs to be normalized between -1 and 1! use max z (arcsin(max/z), or something like that) and max rho for this

In [155]:
# put spherical coords in bodydata matrix for displaying in the model 
# for (shoulder to elbow) and (elbow to wrist) ([:,:,0] == shoulder (empty), [:,:,1] == elbow, [:,:,2] == wrist)
freemocap_3d_body_data[:, 27, :] = rho
freemocap_3d_body_data[:, 28, :] = theta    # 
freemocap_3d_body_data[:, 29, :] = phi      # z / rho^2

### equations pt 2: intro forces equations:

In [156]:
forearm_len = freemocap_3d_body_data[:, 32, 0]
upperarm_length = freemocap_3d_body_data[:, 32, 1]
elbow_angle = freemocap_3d_body_data[:, 32, 2]

rho = freemocap_3d_body_data[:, 27, :] * sim_to_real_conversion_factor
theta = freemocap_3d_body_data[:, 28, :]
phi = freemocap_3d_body_data[:, 29, :]

h_p = 1.78    # meters      # height of person
w_p = 90      # kilograms   # weight of person
w_bal = 3     # kilograms   # weight of ball


In [157]:
# from paper
#phi = np.pi - phi   # equivalent to `180 - phi`

# from email
theta_arm = phi[:, 1] - (np.pi / 2)  # gonna use the elbow for this, since that's representing the shoulder to elbow, or the base of the arm 
w_fa = w_p * 0.023
cgf = h_p * 0.432 * 0.216     # maybe go back and change these ratios, instead of averages use the approximations from earlier? (just a thought)
f = h_p * 0.216
b = h_p * 0.11
u = h_p * 0.173

# referencing paper
theta_fa = theta_arm + elbow_angle
theta_fa_coef = np.sin(theta_fa)    # "sin(theta_u + theta_arm - 90(deg))" from the paper line 4 and 5
theta_u = elbow_angle
theta_b = np.pi - ( (b - u * np.cos(theta_u)) / np.sqrt( (b ** 2) + (u ** 2) - (2 * b * u * np.cos(theta_u)) ) )

la_fa = cgf * theta_fa_coef
la_bal = f * theta_fa_coef
la_bic = b * np.sin(theta_b)

force_bicep = ( (w_fa * la_fa + w_bal * la_bal) / la_bic )

may be no issue, but might wanna think about/account for the fact that the paper uses `sin` just fine because it is flat. this is 3D, so it may need to either use `tan` or a conjunction of `sin` and `cos`.

actually...
- use `sin` for the `z-axis`
- consider using `cos` or `tan` for `x-` and `y- axes`

In [158]:
# prep for visual in plotly output
freemocap_3d_body_data[:, 32, 0] = theta_fa_coef
freemocap_3d_body_data[:, 32, 1] = force_bicep 
freemocap_3d_body_data[:, 32, 2] = np.rad2deg(elbow_angle)

freemocap_3d_body_data[:, 27, :] = rho
freemocap_3d_body_data[:, 28, :] = np.rad2deg(theta) 
freemocap_3d_body_data[:, 29, :] = np.rad2deg(phi)

### Looks like it worked! Now onto 3D plotting, using resources again from the default .ipynb generated with the recording from FreeMoCap...

In [159]:
# saving the body data just in case...
save_body_data = freemocap_3d_body_data

In [162]:
# this cell was copied and pasted straight from the FreeMoCap output .ipynb file

def calculate_axes_means(skeleton_3d_data):
    mx_skel = np.nanmean(skeleton_3d_data[:,0:33,0])
    my_skel = np.nanmean(skeleton_3d_data[:,0:33,1])
    mz_skel = np.nanmean(skeleton_3d_data[:,0:33,2])

    return mx_skel, my_skel, mz_skel

ax_range = 1500

mx_skel, my_skel, mz_skel = calculate_axes_means(freemocap_3d_body_data)

# Create a list of frames
frames = [go.Frame(data=[go.Scatter3d(
    x=freemocap_3d_body_data[i, [11, 13, 15], 0],
    y=freemocap_3d_body_data[i, [11, 13, 15], 1],
    z=freemocap_3d_body_data[i, [11, 13, 15], 2],
    mode='markers',#+text',
    marker=dict(
        size=5,  # Adjust marker size as needed
        color=freemocap_3d_body_data[i, 11:17:2, 1],
        colorscale='Jet',
        opacity=0.8
    )
)], name=str(i),
# add spherical coord annotations -BD
layout = go.Layout(annotations = [
    dict(
        text = "sin(theta_u + theta_arm - 90(deg)), Bicep force, Elbow angle (between upper and fore-arm): ",
        x = 0.1, y = 1,
        showarrow = False
    ),
    dict(
        text = "\t" + str(freemocap_3d_body_data[i, 32, :]),
        x = 0.1, y = 0.9,
        showarrow = False
    ),
    dict(
        text = "Rho: " + str(freemocap_3d_body_data[i, 27, :]),
        x = 0.1, y = 0.8,
        showarrow = False
    ),
    dict(
        text = "Theta: " + str(freemocap_3d_body_data[i, 28, :]),
        x = 0.1, y = 0.7,
        showarrow = False
    ),
    dict(
        text = "Phi: " + str(freemocap_3d_body_data[i, 29, :]),
        x = 0.1, y = 0.6,
        showarrow = False
    ),
])
) for i in range(freemocap_3d_body_data.shape[0])]

# Define axis properties
axis = dict(
    showbackground=True,
    backgroundcolor="rgb(230, 230,230)",
    gridcolor="rgb(255, 255, 255)",
    zerolinecolor="rgb(255, 255, 255)",
)

# Create a figure
fig = go.Figure(
    data=[go.Scatter3d(
        x=freemocap_3d_body_data[0, [11, 13, 15], 0],
        y=freemocap_3d_body_data[0, [11, 13, 15], 1],
        z=freemocap_3d_body_data[0, [11, 13, 15], 2],
        mode='markers',
        marker=dict(
            size=5,  # Adjust marker size as needed
            color=freemocap_3d_body_data[0, 11:17:2, 1],
            colorscale='Jet',
            opacity=0.8
        )
    )],
    layout=go.Layout(
        scene=dict(
            xaxis=dict(axis, range = [400, 1400]),          #range=[mx_skel-ax_range, mx_skel+ax_range]), # Adjust range as needed
            yaxis=dict(axis, range = [-500, 1000]),       #range=[my_skel-ax_range, my_skel+ax_range]), # Adjust range as needed
            zaxis=dict(axis, range = [-1000, 0]),       #range=[mz_skel-ax_range, mz_skel+ax_range]),  # Adjust range as needed
            aspectmode='cube'
        ),
        updatemenus=[dict(
            type='buttons',
            showactive=True,
            buttons=[dict(
                label='Play',
                method='animate',
                args=[None, {"frame": {"duration": 30}}]
            ),
            # add pause button  -BD
            dict(
                args = [[None], {"frame": {"duration": 0, "redraw": False},
                                 "mode": "immediate",
                                 "transition": {"duration": 0}}],
                label = 'Stop',
                method = 'animate'
            )]
        )]
    ),
    frames=frames
)


fig.show()